# De novo assembly with Illumina and Nanopore raw data

This part will explore cutting-edge tools for de novo assembly and hybrid de novo assembly with Illumina and Nanopore data and evaluate the results.

## Illumina data

### What is Paired-End Sequencing?

Paired-end sequencing allows users to sequence both ends of a fragment and generate high-quality, alignable sequence data. Paired-end sequencing facilitates detection of genomic rearrangements and repetitive sequence elements, as well as gene fusions and novel transcripts.

In addition to producing twice the number of reads for the same time and effort in library preparation, sequences aligned as read pairs enable more accurate read alignment and the ability to detect insertion-deletion (indel) variants, which is not possible with single-read data.

### Installing software needed for Part 1

The first software that we need to install its a software that allows us to check the quality of sequencing output or raw reads. To do that we will use FastQC sofwate.

FastQC is already in the repos of Ubuntu 20.04 therefore to install FasQC we just need to:

1.- Update Ubuntu repos:

*>sudo apt-get update*

2.- Install FastQC package

*>sudo apt-get install fastqc*

##note: as you see you need root access to update the repos of your distribution and install the software.

Other option to install software is to use a conda environment.

For this course we will create a conda environment with all the software needed to produce the final assemblies of our strains.

Conda enviroments allow you to have all the software and packages needed in a simple environment. You don't need to have sudo access to create conda environments as they are installed in your home directory.

First we will create an environment with the name "course_assembly" and we will install in this environment all the software needed for this part of the course.

To create the conda environment you just need to type in you shell:

*>conda create --name course_assembly*

With this an empty environment will be created with the name course_assembly

To active the conda environment:

*>activate course_assebly*

Once the environment is activated in your shell you can install last version available of different packages using:

*>conda install -c bioconda fastqc*

*>conda install -c bioconda spades*

*>conda install -c bioconda fastp*

*>conda install -c bioconda trimmomatic*

*>conda install -c bioconda unicycler*



There some software whom owners doesn't recommend to install it using a'package manager' as *canu* software https://github.com/marbl/canu.

In this case we will use directly their precompiled binaries to use the last version of *canu*. https://github.com/marbl/canu/releases. 
The precompiled binares for linux are those showed as .tar.gz.





### Checking quality of paired end Illumina reads

#### FASTQC software

http://www.bioinformatics.babraham.ac.uk/projects/fastqc/
http://manpages.ubuntu.com/manpages/bionic/man1/fastqc.1.html

FastQC aims to provide a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis.

*source activate course_assembly*

*fastqc 57_P3_S5_L001-4_R1_001.fastq.gz 57_P3_S5_L001-4_R2_001.fastq.gz -t 32 -o fastqc_out*

#### Fastp software

https://github.com/OpenGene/fastp


A tool designed to provide fast all-in-one preprocessing for FastQ files. This tool is developed in C++ with multithreading supported to afford high performance.

fastp supports both single-end (SE) and paired-end (PE) input/output. For PE data, you should also specify read2 input by -I or --in2, and specify read2 output by -O or --out2.


*source activate course_assembly*

*mkdir fastp*

*fastp -i 57_P3_S5_L001-4_R1_001.fastq.gz -I 57_P3_S5_L001-4_R2_001.fastq.gz -o ./fastp/57_P3_S5_L001-4_R1_001.fastq_trim.gz -O ./fastp/57_P3_S5_L001-4_R2_001.fastq_trim.gz -h --detect_adapter_for_pe -w 32*


Quality of the trimmed reads can be observed again using FASTQC

*fastqc 57_P3_S5_L001-4_R1_001.fastq_trim.gz 57_P3_S5_L001-4_R2_001.fastq_trim.gz -t 32 -o fastqc_out*


#### Trimmomatic

http://www.usadellab.org/cms/?page=trimmomatic

Trimmomatic is a fast, multithreaded command line tool that can be used to trim and crop Illumina (FASTQ) data as well as to remove adapters. These adapters can pose a real problem depending on the library preparation and downstream application.

There are two major modes of the program: Paired end mode and Single end mode. The paired end mode will maintain correspondence of read pairs and also use the additional information contained in paired reads to better find adapter or PCR primer fragments introduced by the library preparation process.
Trimmomatic works with FASTQ files (using phred + 33 or phred + 64 quality scores, depending on the Illumina pipeline used). Files compressed using either „gzip‟ or „bzip2‟ are supported, and are identified by use of „.gz‟ or „.bz2‟ file extensions.
The current trimming steps are:
    ILLUMINACLIP: Cut adapter and other illumina-specific sequences from the read. SLIDINGWINDOW: Performs a sliding window trimming approach. It starts scanning at the 5‟ end and clips the read once the average quality within the window falls below a threshold.
    MAXINFO: An adaptive quality trimmer which balances read length and error rate to maximise the value of each read
    LEADING: Cut bases off the start of a read, if below a threshold quality
    TRAILING: Cut bases off the end of a read, if below a threshold quality
    CROP: Cut the read to a specified length by removing bases from the end HEADCROP: Cut the specified number of bases from the start of the read MINLEN: Drop the read if it is below a specified length
    AVGQUAL: Drop the read if the average quality is below the specified level TOPHRED33: Convert quality scores to Phred-33
    TOPHRED64: Convert quality scores to Phred-64
    

*java -jar trimmomatic-0.39.jar PE input_forward.fq.gz input_reverse.fq.gz output_forward_paired.fq.gz output_forward_unpaired.fq.gz output_reverse_paired.fq.gz output_reverse_unpaired.fq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads *java -jar trimmomatic-0.39.jar PE input_forward.fq.gz input_reverse.fq.gz output_forward_paired.fq.gz output_forward_unpaired.fq.gz output_reverse_paired.fq.gz output_reverse_unpaired.fq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:15 TRAILING:20 SLIDINGWINDOW:5:20 MINLEN:50 -threads 8*
*

This will perform the following:

Remove adapters (ILLUMINACLIP:TruSeq3-PE.fa:2:30:10)

Remove leading low quality or N bases (below quality 15) (LEADING:15)

Remove trailing low quality or N bases (below quality 20) (TRAILING:20)

Scan the read with a 4-base wide sliding window, cutting when the average quality per base drops below 15 (SLIDINGWINDOW:5:20)

Drop reads below the 36 bases long (MINLEN:50)

Use 8 threads


### Assembly of Illumina reads

#### SPAdes software

https://github.com/ablab/spades
http://cab.spbu.ru/files/release3.13.0/manual.html

SPAdes is not intended for larger genomes (e.g. mammalian size genomes). SPAdes is a very memory intensive program. In multithreaded mode (-t 16), you will want at least 500 Gigabytes if not 1000 Gigabytes of RAM.

SPAdes takes as input paired-end reads, mate-pairs and single (unpaired) reads in FASTA and FASTQ.SPAdes supports mate-pair only assembly.

**Read pair utilization**

While the use of paired reads and mate pairs is key to genome assembly, and not new, SPAdes utilizes so called paired DeBruin graphs to take the advantage of the paired end data. One of the key issues with paired DeBruin graphs is that the resulting genome assemblies do not tolerate variability in insert sizes: The initial formulation of paired DeBruijn graphs assumed constant distance between pairs of reads. In practice this distance is always variable. SPAdes performs k-bimer (these are k-mers derived from paired reads) adjustment to identify exact or nearly-exact distances for each k-bimer pair.

**Error correction**

Sequencing data contains a substantial number of sequencing errors that manifest themselves as deviations (bulges and non-connected components) within the assembly graph. One way to improve the graph before assembly it is to minimize the number of sequencing errors by performing error correction. SPAdes uses BayesHammer (Nikolenko et al. 2013) to correct the reads. Here is a brief summary of what it does:

SPAdes (or rather BayesHammer) counts k-mers in reads and computes k-mer statistics that take into account base quality values.

A Hamming graph is constructed in which k-mers are nodes. In this graph edges connect nodes (k-mers) if they differ from each other by a number of nucleotides up to a certain threshold (the Hamming distance). The graph is central to the error correction algorithm.

Then Bayesian subclustering is done on the graph from the previous step. For each k-mer we now know the center of its subcluster.

Solid k-mers are derived from cluster centers and are assumed to be error free.
Solid k-mers are mapped back to the reads.
Reads are corrected using solid k-mers.

**Code**

*source activate course_assembly*

*mkdir SPAdes*

*spades.py -1 57_P3_S5_L001-4_R1_001.trim.fastq.gz -2 57_P3_S5_L001-4_R2_001.trim.fastq.gz -o ./SPAdes/57 -t 8 --careful*


### Oxford Nanopore reads

#### From FAST5 to FASTQ - Basecalling

Oxford Nanopore machines output data in fast5 format that contains additional information besides sequence data. In this step we will convert the fast5 reads into fastq format.

Base calling is the process of translating the electronic raw signal of the sequencer into bases, i.e., ATCG. As for most bioinformatic tasks there are many different tools to solve this problem. Here, we will only focus on the current state-of-the-art basecaller Guppy, which is the current “official” ONT basecaller. Although basecalling can be performed “live”, i.e., in real time while sequencing, it is often useful to separate the sequencing from basecalling. One advantage of “offline” basecalling is that the basecaller can use significant amounts of compute and read/write resources which may slow the sequencing process and, in rare cases, even lead to loss of sequencing data.

**Guppy**

https://denbi-nanopore-training-course.readthedocs.io/en/latest/basecalling/basecalling.html

Guppy is a neural network based basecaller that in addition to basecalling also performs filtering of low quality reads, clipping of Oxford Nanopore adapters and estimation of methylation probabilities per base.

*source activate course_assembly*

*mkdir guppy_out*

guppy_basecaller –i ./57_fast5 –s ./guppy_out –c dna_r9.4.1_450bps_hac.cfg --num_callers 1 --cpu_threads_per_caller 8 --qscore_filtering --min_qscore 8 --compress_fastq



#### Checking quality of Oxford Nanopore reads

**Nanopack**

https://github.com/wdecoster/nanopack

*NanoPlot -t 8 -p ./guppy --summary ./guppy_out/sequencing_summary.txt --loglength -o summary-plots-log-transformed*

*NanoPlot -t 8 --fastq ./guppy_out/pass/SeranPatient3_s5.pass.pchop.fastq --maxlength 40000 --plots hex dot*


**Nanofilt**


*NanoLyse < ./guppy_out/pass/SeranPatient3_s5.pass.pchop.fastq --reference lambda.fasta | NanoFilt -q 9 > SeranPatient3_s5.pass.pchop.filtq9.fastq*

#### Assembly of Oxford Nanopore Fastq reads using Flye software


https://github.com/fenderglass/Flye

https://github.com/fenderglass/Flye/blob/flye/docs/USAGE.md

**Flye assembler**

Flye is a de novo assembler for single molecule sequencing reads, such as those produced by PacBio and Oxford Nanopore Technologies. It is designed for a wide range of datasets, from small bacterial projects to large mammalian-scale assemblies. The package represents a complete pipeline: it takes raw PacBio / ONT reads as input and outputs polished contigs. Flye also has a special mode for metagenome assembly.

*flye --nano-raw /SeranPatient3_s5.pass.pchop.fastq -g 2.9m -o flye_out -t 8 -i 10 --plasmids*


**Polishing alignment of high error long-reads**

To create the sam files for polishing using Racon we will use **bwa**

*bwa mem -t 36 assembly.fasta ./nanofilt/SeranPatient3_barcode01.pass.pchop.filtq9.fastq > 53_nano_mapping.sam

**Racon** 

https://github.com/isovic/racon

Racon is intended as a standalone consensus module to correct raw contigs generated by rapid assembly methods which do not include a consensus step. The goal of Racon is to generate genomic consensus which is of similar or better quality compared to the output generated by assembly methods which employ both error correction and consensus steps, while providing a speedup of several times compared to those methods. It supports data produced by both Pacific Biosciences and Oxford Nanopore Technologies.

Racon can be used as a polishing tool after the assembly with either Illumina data or data produced by third generation of sequencing. The type of data inputed is automatically detected.

Racon takes as input only three files: contigs in FASTA/FASTQ format, reads in FASTA/FASTQ format and overlaps/alignments between the reads and the contigs in MHAP/PAF/SAM format. Output is a set of polished contigs in FASTA format printed to stdout. All input files can be compressed with gzip (which will have impact on parsing time).

*racon -t8 ./nanofilt/SeranPatient3_barcode01.pass.pchop.filtq9.fastq ./sam_files/53_nano_mapping.sam 

/flye_out/scaffolds.fasta > racon.fasta


**Medaka**

https://github.com/nanoporetech/medaka

medaka is a tool to create a consensus sequence from nanopore sequencing data. This task is performed using neural networks applied from a pileup of individual sequencing reads against a draft assembly. It outperforms graph-based methods operating on basecalled data, and can be competitive with state-of-the-art signal-based methods, whilst being much faster.

Requires only basecalled data. (.fasta or .fastq)

*medaka_consensus -i SeranPatient3_barcode01.pass.pchop.filtq9.fastq -d ./racon/racon.fasta -o medaka_out -t 8 -m r941_min_fast*

For the selection of the model (-m r941_min_high_g303 in the example above) you need to take into account the basecaller used.

Allowed values can be found by running medaka tools list\_models

For example the model named r941_min_fast_g303 should be used with data from MinION (or GridION) R9.4.1 flowcells using the fast Guppy basecaller version 3.0.3. By contrast the model r941_prom_hac_g303 should be used with PromethION data and the high accuracy basecaller (termed "hac" in Guppy configuration files).


**Pilon**

https://github.com/broadinstitute/pilon

https://github.com/broadinstitute/pilon/wiki

Pilon improves draft assemblies by using the information from the original reads aligned to the draft assembly. 

*bwa mem -t 36 medaka_consensus.fasta ./corrected/54_P3_S2_L001-4_R1_001.trim.fastq.00.0_0.cor.fastq.gz ./corrected/54_P3_S2_L001-4_R2_001.trim.fastq.00.0_0.cor.fastq.gz | samtools v
iew -Sb - | samtools sort - > mapping_paired.sorted.bam*

*samtools index mapping_paired.sorted.bam*

*bwa mem -t 36 medaka_consensus.fasta ./corrected/54_P3_S2_L001-4_R_unpaired.00.0_0.cor.fastq.gz | samtools view -Sb - | samtools sort - > mapping_unpaired.sorted.bam*

*samtools index mapping_unpaired.sorted.bam*

*pilon --genome medaka_consensus.fasta --fix all --changes --verbose --threads * --frags mapping_paired.sorted.bam --unpaired mapping_unpaired.sorted.bam --output consensus.pilon.fna*


### Hybrid assembly using Oxford Nanopore and short-reads from Illumina sequencing

**Unicycler**

https://github.com/rrwick/Unicycler

https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005595

Unicycler is an assembly pipeline for **bacterial** genomes. It can assemble Illumina-only read sets where it functions as a SPAdes-optimiser. It can also assembly long-read-only sets (PacBio or Nanopore) where it runs a miniasm+Racon pipeline. For the best possible assemblies, give it both Illumina reads and long reads, and it will conduct a hybrid assembly.

Unicycler is designed specifically for hybrid assembly (that is, using both short- and long-read sequencing data) of small (e.g., bacterial, viral, organellar) genomes. In our hands it has produced complete high quality assemblies. Unicycler employs a multi-step process that utilizes a number of software tools, among them: Spades and Pilon.

**canu assembly as scaffolds for hybrid assembly**

*canu -p 58_canu -d ./canu_out genomeSize=3m -nanopore-raw SeranPatient3_s6.pass.pchop.filtq9.fastq useGrid=false batThreads=8 batMemory=32 > canu.log*

**Unicycler assembly using Canu long-reads scaffolds and Illumina reads**

*unicycler -1 ./SPAdes/58/corrected/58_P3_S6_L001-4_R1_001.trim.fastq.00.0_0.cor.fastq.gz -2 ./SPAdes/58/corrected/58_P3_S6_L001-4_R2_001.trim.fastq.00.0_0.cor.fastq.gz -l ./canu/58/58_canu.contigs.fasta -o /storage/zudominguez/VREF/53_58/assembly/unicycler_canu/58/ -t 8 --keep 2 --spades_tmp_dir /storage/zudominguez/VREF/assembly/unicycler_mix/58/tmp --existing_long_read_assembly ./canu/58/58_canu.contigs.gfa*


### Assembly quality comparisson

Mummer - alignment

QUAST